# Implémentation de méthodes élémentaires pour la classification supervisée : ADL, ADQ, Naive Bayes et classifieur par plus proches voisins

Nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix

from scipy.io import arff

Le jeu de données [Vertebral Column](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column) permet d'étudier les pathologies d'hernie discale et de Spondylolisthesis. Ces deux pathologies sont regroupées dans le jeu de données en une seule catégorie dite `Abnormale`. 

Il s'agit donc d'un problème de classification supervisée à deux classes :
- Normale (NO) 
- Abnormale (AB)    

avec 6 variables bio-mécaniques disponibles (features).

L'objectif du TP est d'implémenter quelques méthodes simples de classification supervisée pour ce problème.

# Importation des données

> Télécharger le fichier column_2C.dat depuis le site de l'UCI à [cette adresse](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column). 
>
> On peut importer les données sous python par exemple avec la librairie [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html). Vous pourrez au besoin consulter la documentation de la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). 
> 
> Le chemin donné dans la fonction `read_csv`est une chaîne de caractère qui spécifie le chemin complet vers le ficher sur votre machine. On peut aussi donner une adresse url si le fichier est disponible en ligne.
>
> Attention à la syntaxe pour les chemins sous Windows doit etre de la forme  `C:/truc/machin.csv`. 
> 
> Voir ce [blog](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) pour en savoir plus sur la "manipulation des chemins" sur des OS variés. 

In [ ]:
file_path= r"C:\Users\kassi\Downloads\vertebral+column\column_2C_weka.arff"
data, meta = arff.loadarff(file_path)
Vertebral = pd.DataFrame(data)
Vertebral.columns = ["pelvic_incidence",
                                       "pelvic_tilt",
                                       "lumbar_lordosis_angle",
                                       "sacral_slope",
                                       "pelvic_radius",
                                       "degree_spondylolisthesis",
                                       "class"]
Vertebral["class"] = Vertebral["class"].apply( lambda x: "Abnormal" if x==b'Abnormal' else 'Normal')

> Vérifier à l'aide des méthodes `.head()`  et `describe()` que les données sont bien importées.

In [ ]:
Vertebral.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Abnormal


In [ ]:
Vertebral.describe()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,60.496653,17.542822,51.930930,42.953831,117.920655,26.296694
std,17.236520,10.008330,18.554064,13.423102,13.317377,37.559027
min,26.147921,-6.554948,14.000000,13.366931,70.082575,-11.058179
25%,46.430294,10.667069,37.000000,33.347122,110.709196,1.603727
50%,58.691038,16.357689,49.562398,42.404912,118.268178,11.767934
75%,72.877696,22.120395,63.000000,52.695888,125.467674,41.287352
max,129.834041,49.431864,125.742385,121.429566,163.071041,418.543082


> Les librairies de Machine Learning telles que `sckitlearn` prennent en entrée des tableau numpy (pas des objets pandas). Créer un tableau numpy que vous nommerez `VertebralVar` pour les features et un vecteur numpy `VertebralClas` pour la variable de classe. Voir par exemple [ici](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy).

In [ ]:
cols = list(Vertebral.columns)
cols.pop()
VertebralVar = Vertebral[cols]
VertebralClas = Vertebral['class']
print(VertebralVar)
print('\n\n')
VertebralClas

     pelvic_incidence  pelvic_tilt  lumbar_lordosis_angle  sacral_slope  \
0           63.027817    22.552586              39.609117     40.475232   
1           39.056951    10.060991              25.015378     28.995960   
2           68.832021    22.218482              50.092194     46.613539   
3           69.297008    24.652878              44.311238     44.644130   
4           49.712859     9.652075              28.317406     40.060784   
..                ...          ...                    ...           ...   
305         47.903565    13.616688              36.000000     34.286877   
306         53.936748    20.721496              29.220534     33.215251   
307         61.446597    22.694968              46.170347     38.751628   
308         45.252792     8.693157              41.583126     36.559635   
309         33.841641     5.073991              36.641233     28.767649   

     pelvic_radius  degree_spondylolisthesis  
0        98.672917                 -0.254400  
1    

0      Abnormal
1      Abnormal
2      Abnormal
3      Abnormal
4      Abnormal
         ...   
305      Normal
306      Normal
307      Normal
308      Normal
309      Normal
Name: class, Length: 310, dtype: object

# Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 60% des observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de test. 

In [ ]:
from sklearn.model_selection import train_test_split
VertebralVar_train,VertebralVar_test,VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar,VertebralClas,train_size=0.6)
ntot = len(VertebralVar) ### longueur totale de l'échantillon -  TO DO ####
ntrain = len(VertebralVar_train) ### longueur totale de l'échantillon d'apprentissage - TO DO ####
ntest = len(VertebralVar_test) ### longueur totale de l'échantillon de test -TO DO ####
print(ntot, ntrain, ntest)

310 186 124


Remarque : on peut aussi le faire à la main avec la fonction [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

# Extraction des deux classes

> Extraire les deux sous-échantillons de classes respectives "Abnormale" et "Normale" pour les données d'apprentissage et de test.

In [ ]:
train_abnormal = VertebralVar_train[VertebralClas_train == "Abnormal"]
train_normal   = VertebralVar_train[VertebralClas_train == "Normal"]

In [ ]:
n_AB = len(train_abnormal)
n_NO = len(train_normal)
print(n_AB)
print(n_NO)

129
57


# Analyse Discriminante Linéaire (LDA)

Nous allons ajuster à la main le classifieur de l'analyse discriminante linéaire sur l'échantillon d'apprentissage et ensuite évaluer ses performances en considérant ses prédictions sur l'échantillon de test.

- Pour calculer la matrice de covariance on peut utiliser 
par exemple la fonction [`empirical_covariance`](https://scikit-learn.org/stable/modules/generated/sklearn.covariance.empirical_covariance.html#sklearn.covariance.empirical_covariance) de la librairie  [`sklearn.covariance`](http://scikit-learn.org/stable/modules/covariance.html).
- Pour calculer la valeur de la densité d'une gaussienne multidimensionnelle en un point $x$ de $\mathbb R ^d$ on peut utililser la fonction [`multivariate_normal`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) de la librairie [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html). 

D'abord on calcule le centre et la matrice de covariance pour chacun des deux groupes  :

In [ ]:
from sklearn.covariance import empirical_covariance
Cov_AB = empirical_covariance(train_abnormal)
Cov_NO = empirical_covariance(train_normal)
mean_AB = train_abnormal.mean()
mean_NO = train_normal.mean()
print(Cov_AB,'\n\n',Cov_NO,'\n\n')

[[ 309.70821206   91.56207438  219.6878246   218.14613768  -36.20270589
   525.85945177]
 [  91.56207438  102.12271802   66.02361286  -10.56064363   26.43508417
   121.84344301]
 [ 219.6878246    66.02361286  361.36428688  153.66421173   14.48896562
   398.95628611]
 [ 218.14613768  -10.56064363  153.66421173  228.70678131  -62.63779005
   404.01600877]
 [ -36.20270589   26.43508417   14.48896562  -62.63779005  208.63582552
    73.92416486]
 [ 525.85945177  121.84344301  398.95628611  404.01600877   73.92416486
  2142.70990972]] 

 [[184.3710931   62.55143594 137.46485801 121.81965716 -64.37421199
   15.45968662]
 [ 62.55143594  51.63430587  37.55259059  10.91713007 -26.17158551
    7.16273523]
 [137.46485801  37.55259059 159.24374669  99.91226742 -26.87049165
   13.00865171]
 [121.81965716  10.91713007  99.91226742 110.90252709 -38.20262648
    8.29695139]
 [-64.37421199 -26.17158551 -26.87049165 -38.20262648  85.06361507
   -1.33933807]
 [ 15.45968662   7.16273523  13.00865171   8.29

In [ ]:
print(mean_AB)
print(mean_NO)

pelvic_incidence             63.728982
pelvic_tilt                  19.271044
lumbar_lordosis_angle        54.547379
sacral_slope                 44.457938
pelvic_radius               116.072614
degree_spondylolisthesis     38.560392
dtype: float64
pelvic_incidence             52.742588
pelvic_tilt                  13.434078
lumbar_lordosis_angle        43.695151
sacral_slope                 39.308509
pelvic_radius               122.758333
degree_spondylolisthesis      2.164741
dtype: float64


In [ ]:
Puis on calcule la matrice de covariance intra (voir le cours ...)

In [ ]:
Intra_Cov = (n_AB/ntot)*Cov_AB + (n_NO/ntot)*Cov_NO
print(Intra_Cov)

[[162.77906988  49.60303046 116.69427832 113.17603941 -26.90154562
  221.66797231]
 [ 49.60303046  51.99027761  34.3791733   -2.38724714   6.18821124
   52.01961309]
 [116.69427832  34.3791733  179.6544728   82.31510502   1.08857594
  168.40920663]
 [113.17603941  -2.38724714  82.31510502 115.56328656 -33.08975686
  169.64835923]
 [-26.90154562   6.18821124   1.08857594 -33.08975686 102.46015339
   30.5157258 ]
 [221.66797231  52.01961309 168.40920663 169.64835923  30.5157258
  897.8222584 ]]


Pour une observation $x \in \mathbb R^6$ (décrite par ses 6 features), la régle du MAP (Maximum A Posteriori) dans le cas de l'analyse discriminante linéaire consiste à choisir la catégorie $\hat y (x) = \hat k $ qui maximise (en $k$) 
$$ score_k(x) = \hat \pi_k \hat f_k(x) $$
où :
- $k$ est le numéro de la classe ;
- $\hat \pi_k$ est la proportion observée de la classe $k$, 
- $\hat f_k$ est la densité gaussienne multidimensionnelle de la classe $k$ (avec pour paramètre de centrage $\mu_k$ et pour matrice de covariance Intra (pour toutes les classes).

On calcule tout d'abord pour toutes les données de test les valeurs des scores sur les deux catégories :

In [ ]:
score_LDA_test = [   [np.transpose(x).dot(np.linalg.inv(Cov_AB)).dot(mean_AB) + np.log(n_AB/ntot)-0.5*np.transpose(mean_AB).dot(np.linalg.inv(Cov_AB)).dot(mean_AB) , np.transpose(x).dot(np.linalg.inv(Cov_NO)).dot(mean_NO) + np.log(n_NO/ntot)-0.5*np.transpose(mean_NO).dot(np.linalg.inv(Cov_NO)).dot(mean_NO)    ]  for x in VertebralVar_test.values]
score_LDA_test

[[np.float64(76.26468012164483), np.float64(204.80135574252068)],
 [np.float64(43.914192623063755), np.float64(153.99042108715236)],
 [np.float64(67.74030713070319), np.float64(219.74544781506717)],
 [np.float64(57.136191122551864), np.float64(178.5117268121308)],
 [np.float64(82.10517491595024), np.float64(200.18522679866015)],
 [np.float64(55.4733070567287), np.float64(198.79895071745574)],
 [np.float64(64.65295152141847), np.float64(211.11544893742263)],
 [np.float64(68.90040303891661), np.float64(273.86989173353373)],
 [np.float64(55.710484230743205), np.float64(155.06676240907706)],
 [np.float64(60.96914465343947), np.float64(205.84406059295594)],
 [np.float64(68.50716354368856), np.float64(161.70357830882728)],
 [np.float64(53.09509055581806), np.float64(162.23845902145564)],
 [np.float64(59.08643379059289), np.float64(164.20208074957787)],
 [np.float64(58.36854807901331), np.float64(159.68692969218552)],
 [np.float64(60.836380912600035), np.float64(182.66123758316695)],
 [np.flo

et on choisit la classe qui maximise le score (pour chaque élement des données de test):

In [ ]:
pred_LDA_test = []
for k in range (len(score_LDA_test)):
    if score_LDA_test[k][0]>score_LDA_test[k][1] :
        pred_LDA_test.append(['Abnormal'])
    else :
        pred_LDA_test.append(['Normal'])
pred_LDA_test
VertebralClas_test.values == np.array(pred_LDA_test).ravel()

array([False, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True,  True,  True, False, False,  True,
       False,  True, False,  True,  True, False,  True, False, False,
       False, False,  True,  True, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True, False,  True, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True, False, False, False, False, False,  True, False,
        True, False, False,  True, False, False, False, False,  True,
       False, False, False,  True, False,  True,  True, False,  True,
       False,  True, False, False, False, False,  True, False, False,
        True, False, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True, False,  True, False, False])

La matrice de confusion est une matrice qui synthétise les performances d'une régle de classification. Chaque ligne correspond à une classe réelle, chaque colonne correspond à une classe estimée. La cellule (ligne L, colonne C) contient le nombre d'éléments de la classe réelle L qui ont été estimés comme appartenant à la classe C. Voir par exemple [ici](https://fr.wikipedia.org/wiki/Matrice_de_confusion).


> Evaluer les performances de la méthode sur l'échantillon test. Vous pourrez utiliser la fonction [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) de la librairie [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [ ]:
from sklearn.metrics import confusion_matrix
vraie_classe_test = VertebralClas_test 
cnf_matrix_LDA_test = confusion_matrix(np.array(vraie_classe_test).ravel(), np.array(pred_LDA_test).ravel(), labels=['Normal', 'Abnormal'])
cnf_matrix_LDA_test.astype('float') / cnf_matrix_LDA_test.sum(axis=1).reshape(-1,1) 

array([[1.        , 0.        ],
       [0.98765432, 0.01234568]])

In [ ]:
len(train_normal)

57

> Vérifier que l'évaluation de la méthode sur les données d'apprentissage donne un résulat sensiblement plus optimiste.

In [ ]:
score_LDA_train =  [[np.transpose(x).dot(np.linalg.inv(Cov_AB)).dot(mean_AB) + np.log(n_AB/ntot)-0.5*np.transpose(mean_AB).dot(np.linalg.inv(Cov_AB)).dot(mean_AB) , np.transpose(x).dot(np.linalg.inv(Cov_NO)).dot(mean_NO) + np.log(n_NO/ntot)-0.5*np.transpose(mean_NO).dot(np.linalg.inv(Cov_NO)).dot(mean_NO)    ]  for x in VertebralVar_train.values]
pred_LDA_train = []
for k in range (len(score_LDA_train)):
    if score_LDA_train[k][0]>score_LDA_train[k][1] :
        pred_LDA_train.append(['Abnormal'])
    else :
        pred_LDA_train.append(['Normal'])
vraie_classe_train = np.array(VertebralClas_train).ravel()
cnf_matrix_LDA_train =  confusion_matrix(vraie_classe_train,pred_LDA_train,labels=['Normal','Abnormal'])
cnf_matrix_LDA_train.astype('float') / cnf_matrix_LDA_train.sum(axis=1).reshape(-1,1) 

array([[1.        , 0.        ],
       [0.99224806, 0.00775194]])

Il existe bien sûr une fonction scikit-learn pour la méthode LDA : voir [ici](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html).

# Regression logistique

> Ajuster un modèle de régression logistique sur les données d'apprentissage en utilisant la fonction  `sklearn.linear_model.LogisticRegression`, voir [ici](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).   
> Par défaut une penalty $l_2$ est présente dans la fonction de perte (voir la doc). Pour obtenir un ajustement pour la regression logistique standard, il faut donc  prendre garder à préciser `penalty=‘none’` dans les options de la fonction.

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train = VertebralVar_train.values
y_train = np.ravel(VertebralClas_train)

logreg = LogisticRegression(max_iter = 1000, penalty = None).fit(X_train,y_train)

> Afficher les coefficients estimés de la regression logistique 

> Afficher avec la fonction `predict_proba` les estimations des probabilités a posteriori sur l'échantillon de test.

> Afficher la matrice de confusion et comparer avec les résultats avec ceux de la LDA

In [ ]:
X_test = VertebralVar_test.values
y_test = np.ravel(VertebralClas_test)
pred_log_test = logreg.predict(X_test)
cnf_matrix_LDA_train =  confusion_matrix(y_test,pred_log_test,labels=['Normal','Abnormal'])
cnf_matrix_LDA_train.astype('float') / cnf_matrix_LDA_train.sum(axis=1).reshape(-1,1) 

array([[0.8372093 , 0.1627907 ],
       [0.09876543, 0.90123457]])

In [ ]:
X_test = VertebralVar_train.values
y_test = np.ravel(VertebralClas_train)
pred_log_test = logreg.predict(X_test)
cnf_matrix_LDA_train =  confusion_matrix(y_test,pred_log_test,labels=['Normal','Abnormal'])
cnf_matrix_LDA_train.astype('float') / cnf_matrix_LDA_train.sum(axis=1).reshape(-1,1) 

array([[0.71929825, 0.28070175],
       [0.10852713, 0.89147287]])

> **Bonus** (à faire à la fin si vous le souhaitez) : recoder par vous-même un prédicteur de régression logistique et comparer vos prévisions avec celles obtenues par la fonction de sklearn. Notez que sur ce jeu de données l'estimation du modèle de régression logistique est relativement instable du fait que la fonction de vraisembance correspondante est assez ``plate".

# Analyse Discriminante Quadratique (QDA)

> Reprendre et adapter les codes précédents (de la LDA) pour ajuster cette fois un classifieur par analyse discriminante quadratique sur les données d'apprentissage. 
> 
> Evaluer la qualité du classifieur sur les données de test.

In [ ]:
score_QDA_test = [
    [ np.log(n_AB/ntot) - 0.5 * np.transpose(x - mean_AB).dot(np.linalg.inv(Cov_AB)).dot(x - mean_AB) - 0.5 * np.log(abs(np.linalg.det(Cov_AB))),
         np.log(n_NO/ntot) - 0.5 * np.transpose(x - mean_NO).dot(np.linalg.inv(Cov_NO)).dot(x - mean_NO) - 0.5 * np.log(abs(np.linalg.det(Cov_NO)))
    ] for x in VertebralVar_test.values   ] # <-- CORRECTION ICI

# Prédictions
pred_QDA_test = []
for k in range(len(score_QDA_test)):
    if score_QDA_test[k][0] > score_QDA_test[k][1]:
        pred_QDA_test.append('Abnormal')   # <-- PAS de crochets
    else:
        pred_QDA_test.append('Normal')

# Vérification des dimensions
print(len(VertebralClas_test), len(pred_QDA_test))  # doivent être égales

# Matrice de confusion
cnf_matrix_QDA_test = confusion_matrix( np.array(VertebralClas_test).ravel(), np.array(pred_QDA_test).ravel(),  labels=['Normal', 'Abnormal'])

# Normalisation ligne par ligne
cnf_matrix_QDA_test_norm = (cnf_matrix_QDA_test.astype('float')/ cnf_matrix_QDA_test.sum(axis=1).reshape(-1, 1))

print("Matrice de confusion QDA :\n", cnf_matrix_QDA_test)
print("\nMatrice normalisée :\n", cnf_matrix_QDA_test_norm)

124 124
Matrice de confusion QDA :
 [[38  5]
 [17 64]]

Matrice normalisée :
 [[0.88372093 0.11627907]
 [0.20987654 0.79012346]]


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report

lda = LinearDiscriminantAnalysis()
lda.fit(VertebralVar_train, VertebralClas_train)

pred_LDA = lda.predict(VertebralVar_test)
cnf_matrix_LDA = confusion_matrix(VertebralClas_test, pred_LDA, labels=['Normal', 'Abnormal'])
cnf_matrix_LDA_test_norm = (cnf_matrix_LDA_test.astype('float')/ cnf_matrix_LDA_test.sum(axis=1).reshape(-1, 1))

print("Matrice de confusion LDA :\n", cnf_matrix_LDA_test_norm)

#print("\nRapport de classification LDA :\n")
#print(classification_report(VertebralClas_test, pred_LDA))

Matrice de confusion LDA :
 [[1.         0.        ]
 [0.98765432 0.01234568]]


In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(VertebralVar_train, VertebralClas_train)

pred_QDA = qda.predict(VertebralVar_test)
cnf_matrix_QDA = confusion_matrix(VertebralClas_test, pred_QDA, labels=['Normal', 'Abnormal'])
cnf_matrix_QDA_test_norm = (cnf_matrix_QDA_test.astype('float')/ cnf_matrix_QDA_test.sum(axis=1).reshape(-1, 1))

print("Matrice de confusion QDA :\n", cnf_matrix_QDA_test_norm)


Matrice de confusion QDA :
 [[0.88372093 0.11627907]
 [0.20987654 0.79012346]]


C:\Users\kassi\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
C:\Users\kassi\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(


In [ ]:
x = VertebralVar_train.iloc[[2]]
qda.predict(x)
#VertebralClas_train

array(['Abnormal'], dtype=object)

Il existe bien sûr une fonction scikit-learn pour la méthode QDA : voir [ici](
https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis).

# Gaussian Naive Bayes

Nous allons maintenant ajuster un classifieur naif bayesien sur les données d'apprentissage.

Pour une observation $x \in \mathbb R^6$, la régle du MAP consiste cette fois à choisir la catégorie $\hat y (x) = \hat k $ qui maximise (en $k$) 
$$ score_k(x) = \hat \pi_k \prod_{j=1} ^6  \hat f_{k,j}(x_j)   $$
où :
- $k$ est le numéro de la classe ;
- $\hat \pi_k$ est la proportion observée de la classe $k$, 
- $\hat f_{k,j} $ est la densité gaussienne univariée de la classe $k$ pour la variable $j$. Les paramètres de cette loi valent (ajustés par maximum de vraisemblance) :
    - $\hat \mu_{k,j}$ : la moyenne empirique de la variable $X^j$ restreinte à la classe k,
    - $ \hat \sigma^2_{k,j}$ : la variance empirique de la variable $X^j$ restreinte à la classe k.
    
Noter que la fonction $x \mapsto  \prod_{j=1} ^6  f_{k,j}(x_j) $ peut aussi être vue comme une densité gaussienne multidimensionnelle de moyenne $(\mu_{k,1}, \dots, \mu_{k,6})$ et de matrice de covariance diagonale $diag(\hat \sigma^2_{k,1},\dots,\hat  \sigma^2_{k,6})$. Cette remarque évite de devoir calculer le produit de 6 densités univariées, à la place on calcule plus directement la valeur de la densité multidimensionnelle.

Calcul des moyennes et des variances de chaque variable pour chacun des deux groupes :

In [ ]:
mean_AB = train_abnormal.mean(axis=0)
mean_NO = train_normal.mean(axis=0)

# variances estimées coord par coord pour AB (sur le train) :
var_AB = train_abnormal.var(axis=0)
# variances estimées coord par coord pour NO (sur le train) :
var_NO = train_normal.var(axis=0)

# on forme les matrices de covariance (matrices diagonales car indep) :
Cov_NB_AB =  empirical_covariance(train_abnormal) 
Cov_NB_NO =  empirical_covariance(train_normal)

print(list(mean_AB),'\n\n',mean_NO,'\n\n',var_AB,'\n\n',var_NO,'\n\n',Cov_NB_AB,'\n\n',Cov_NB_NO)

[63.72898208193798, 19.271044487263566, 54.54737868790698, 44.45793759449613, 116.0726138591473, 38.56039164388372] 

 pelvic_incidence             52.742588
pelvic_tilt                  13.434078
lumbar_lordosis_angle        43.695151
sacral_slope                 39.308509
pelvic_radius               122.758333
degree_spondylolisthesis      2.164741
dtype: float64 

 pelvic_incidence             312.127807
pelvic_tilt                  102.920552
lumbar_lordosis_angle        364.187445
sacral_slope                 230.493553
pelvic_radius                210.265793
degree_spondylolisthesis    2159.449831
dtype: float64 

 pelvic_incidence            187.663434
pelvic_tilt                  52.556347
lumbar_lordosis_angle       162.087385
sacral_slope                112.882929
pelvic_radius                86.582608
degree_spondylolisthesis     34.202174
dtype: float64 

 [[ 309.70821206   91.56207438  219.6878246   218.14613768  -36.20270589
   525.85945177]
 [  91.56207438  102.12271802 

Calcul du "score" sur chaque groupe pour chaque element des données test : 

In [ ]:
# Matrices de covariance diagonales (les variances seules suffisent pour Naïve Bayes)
Cov_NB_AB = np.diag(var_AB)
Cov_NB_NO = np.diag(var_NO)

# Scores Naïve Bayes sur le train
score_NB_train = [
    [
        (n_AB / ntot) * np.prod(1 / np.sqrt(2*np.pi*var_AB) * np.exp(-0.5 * ((np.array(x) - mean_AB)/np.sqrt(var_AB))**2)),
        (n_NO / ntot) * np.prod(1 / np.sqrt(2*np.pi*var_NO) * np.exp(-0.5 * ((np.array(x) - mean_NO)/np.sqrt(var_NO))**2))
    ]
    for x in VertebralVar_test.values
]

# Prédiction
pred_NB_train = ['Abnormal' if s[0] > s[1] else 'Normal' for s in score_NB_train]
pred_NB_train


['Abnormal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Normal',
 'Normal',
 'Normal',
 'Abnormal',
 'Normal',
 'Normal',
 'Normal',
 'Abnormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Abnormal',
 'Normal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Normal',
 'Abnormal',
 'Normal',
 'Normal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Normal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Normal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Normal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Abnormal',
 'Normal',
 'Abnormal',
 'Abnormal

In [ ]:
len(np.array(VertebralClas_test).ravel())

124

In [ ]:
len(np.array(pred_NB_train).ravel())

124

Affichage de la matrice de confusion sur les données de test :

In [ ]:
cnf_matrix_NB_test = confusion_matrix(np.array(VertebralClas_test).ravel(), np.array(pred_NB_train).ravel(),labels=['Normal','Abnormal'])
cnf_matrix_NB_test.astype('float') / cnf_matrix_NB_test.sum(axis=1).reshape(-1,1) 

array([[0.90697674, 0.09302326],
       [0.2345679 , 0.7654321 ]])

>  Il existe bien sûr une fonction scikit-learn  pour la méthode Naive Bayes : voir [ici](http://scikit-learn.org/stable/modules/naive_bayes.html). Vérifier que votre prédicteur donne la même réponse de cette fonction.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(VertebralVar_train,VertebralClas_train)
y = gnb.predict(VertebralVar_test)
cnf_matrix_NB_test = confusion_matrix(VertebralClas_test,y,labels=['Normal','Abnormal'])
cnf_matrix_NB_test.astype('float') / cnf_matrix_NB_test.sum(axis=1).reshape(-1,1) 

array([[0.90697674, 0.09302326],
       [0.2345679 , 0.7654321 ]])

# Classifieur par plus proches voisins

Il est préférable d'utiliser la structure de données de type [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) pour effectuer des requêtes de plus proches voisins dans un nuage de points. 

> Contruction du k-d tree pour les données train (pour la métrique euclidienne) :

In [ ]:
from sklearn.neighbors import KDTree
tree =  ### TO DO ####

> Rechercher les 10 plus proches voisins dans les données d'apprentissage du premier point des données de test et afficher les classes de ces observations voisines.

In [ ]:
indices_voisins =  tree.query(### TO DO ####)
print(indices_voisins)
classes_voisins = ### TO DO ####
print(classes_voisins)    

Pour le classifieur par plus proches vosins, la prediction est la classe majoritaire des k plus proches voisins.

> Donner la prédiction pour le premier point de test par vote majoritaire sur ses 10 plus proches voisins 

> Donner la prediction du classifieur ppv pour toutes les données de test. Evaluer la qualité du classifieur.

In [ ]:
k_class = ### CHOISIR  ####  #nombre de plus proche voisins utilisés
pred_kNN_test =  ### TO DO ####
cnf_matrix_kNN =### TO DO ####
cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1).reshape(-1,1) 

Il existe bien sûr une fonction scikit-learn pour le classifieur plus proche voisin, voir [ici](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).